In [2]:
import argparse
import pandas as pd 
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torcheval.metrics.functional import multiclass_f1_score
from torchvision import models

/ext3/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Dataset class to create dataloader for train, val and test
class MyDataset(Dataset):
    def __init__(self, images, labels):
        # Convert images and labels to PyTorch tensors
        self.images = torch.tensor(images, dtype=torch.float32)
        # self.images = self.images.view(batch_size, num_channels, 50, 50)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# Assemble datasets with dataloader
def get_dataset(batch_size, train_images, train_labels, val_images, val_labels):
    
    train_dataset = MyDataset(images = train_images, labels = train_labels) 
    val_dataset = MyDataset(images = val_images, labels= val_labels)

    train_loader = DataLoader(
      train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    val_loader = DataLoader(
      val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_dataset, train_loader, val_dataset, val_loader

In [4]:
def update_model_layers(model, num_channels, num_classes):
    num_ftrs = model.fc.in_features

    # Change size of initial layer to match input size of satelite imagery
    model.conv1 = nn.Conv2d(num_channels, model.conv1.out_channels,
                                kernel_size=model.conv1.kernel_size,
                                stride=model.conv1.stride,
                                padding=model.conv1.padding,
                                bias=False)

    # change size of final output layer of model
    model.fc = nn.Linear(num_ftrs, num_classes)

    return model

In [6]:
batch_size = 32
train_images = np.load('train_humans.npz')['arr_0']
train_labels = np.load('train_labels_humans.npz')['arr_0']
val_images = np.load('val_humans.npz')['arr_0']
val_labels = np.load('val_labels_humans.npz')['arr_0']
num_classes = len(np.unique(val_labels))
num_channels = train_images.shape[1]

human_train_dataset, human_train_loader, human_val_dataset, human_val_loader = get_dataset(batch_size, train_images, train_labels, val_images, val_labels)

In [10]:
# Bring in model
model = torch.hub.load('moskomule/senet.pytorch','se_resnet20', num_classes=num_classes)
model = update_model_layers(model, num_channels, num_classes)

model.load_state_dict(torch.load('dual_split/model:se_resnet20_epoch:3_lr:0.03_mom:0.9_step:15_gamma:0.3_valloss:2.9_f1loss:0.72_batchsize:32_binary:humans.pt'))


Using cache found in /home/mcs9834/.cache/torch/hub/moskomule_senet.pytorch_master


<All keys matched successfully>

In [13]:
model.eval()
validation_loss = 0
correct = 0
y_true = []
y_pred = []
for data, target in human_val_loader:
    y_true.append(target)
    #change data sizing here if it comes in in the correct size
    output = model(data)#.view(batch_size, 98, 50, 50)
    validation_loss += F.cross_entropy(output, target, reduction='sum')
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    y_pred.append(torch.flatten(pred))
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

# change number of classes if switching to binary classification
y_true = torch.cat(y_true)
y_pred = torch.cat(y_pred)
f1 = multiclass_f1_score(y_true, y_pred, average='weighted', num_classes = num_classes)
validation_loss /= len(human_val_loader.dataset)
print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    validation_loss, correct, len(human_val_loader.dataset),
    100. * correct / len(human_val_loader.dataset)))
print('val_loss', validation_loss.item(), 'f1_loss:', f1.item())



Validation set: Average loss: 2.9038, Accuracy: 2723/4864 (56%)

val_loss 2.9038045406341553 f1_loss: tensor(0.7158)


In [16]:
y_pred_humans = y_pred.detach().clone()
y_pred_humans

tensor([1, 1, 1,  ..., 1, 1, 1])

In [17]:
batch_size = 32
train_images = np.load('train_electricity.npz')['arr_0']
train_labels = np.load('train_labels_electricity.npz')['arr_0']
val_images = np.load('val_electricity.npz')['arr_0']
val_labels = np.load('val_labels_electricity.npz')['arr_0']
num_classes = len(np.unique(val_labels))
num_channels = train_images.shape[1]

elec_train_dataset, elec_train_loader, elec_val_dataset, elec_val_loader = get_dataset(batch_size, train_images, train_labels, val_images, val_labels)

In [57]:
# Bring in model
num_classes = len(np.unique(val_labels))
model_elec = torch.hub.load('moskomule/senet.pytorch','se_resnet20', num_classes=num_classes)
model_elec = update_model_layers(model, num_channels, num_classes)

model_elec.load_state_dict(torch.load('dual_split/model:se_resnet20_epoch:1_lr:0.03_mom:0.9_step:15_gamma:0.3_valloss:0.08_f1loss:0.99_batchsize:32_binary:elec.pt'))


Using cache found in /home/mcs9834/.cache/torch/hub/moskomule_senet.pytorch_master


<All keys matched successfully>

In [19]:
model_elec.eval()
validation_loss = 0
correct = 0
y_true = []
y_pred = []
for data, target in elec_val_loader:
    y_true.append(target)
    #change data sizing here if it comes in in the correct size
    output = model_elec(data)#.view(batch_size, 98, 50, 50)
    validation_loss += F.cross_entropy(output, target, reduction='sum')
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    y_pred.append(torch.flatten(pred))
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

# change number of classes if switching to binary classification
y_true = torch.cat(y_true)
y_pred = torch.cat(y_pred)
f1 = multiclass_f1_score(y_true, y_pred, average='weighted', num_classes = num_classes)
validation_loss /= len(elec_val_loader.dataset)
print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    validation_loss, correct, len(elec_val_loader.dataset),
    100. * correct / len(elec_val_loader.dataset)))
print('val_loss', validation_loss.item(), 'f1_loss:', f1.item())


Validation set: Average loss: 0.0767, Accuracy: 4789/4864 (98%)

val_loss 0.07669061422348022 f1_loss: 0.9922304153442383


In [20]:
y_pred_elec = y_pred.detach().clone()
y_pred_elec

tensor([0, 0, 0,  ..., 0, 0, 0])

In [30]:
y_pred_dual_split = np.zeros(len(y_pred_humans))
y_pred_dual_split.fill(5) #making these 5 to ensure only 0s and 1s get filled and not left over

for i in range(len((y_pred_humans))): 
    if y_pred_humans == 1 and y_pred_elec == 0: 
        y_pred_dual_split[i] = 0
    elif y_pred_humans == 0 and y_pred_elec == 0:
        y_pred_dual_split[i] = 1
    elif y_pred_humans == 1 and y_pred_elec == 1:
        y_pred_dual_split[i] = 2
    elif y_pred_humans == 0 and y_pred_elec == 1:
        y_pred_dual_split[i] = 3


array([0., 0., 0., ..., 0., 0., 0.])

In [50]:
# Need 2 sats val loader to check for comparison
val_labels_all = torch.from_numpy(np.load('val_labels2.npz')['labels'])
val_labels_all

tensor([1, 1, 1,  ..., 0, 1, 1])

In [51]:
y_pred_dual_split = torch.from_numpy(y_pred_dual_split.astype('int64'))
y_pred_dual_split

tensor([0, 0, 0,  ..., 0, 0, 0])

In [54]:
# Num classes = 4
num_classes = 4
f1 = multiclass_f1_score(val_labels_all, y_pred_dual_split, average='weighted', num_classes = num_classes)
f1.item()

0.7094720005989075

# Checking dual model - 2sats data for each model

In [6]:
batch_size = 32
val_images = np.load('val_data_2sats.npz')['arr_0']
val_labels_human = np.load('val_labels_humans.npz')['arr_0']
val_labels_elec = np.load('val_labels_electricity.npz')['arr_0']

val_dataset_human = MyDataset(images = val_images, labels= val_labels_human)
val_dataset_elec = MyDataset(images = val_images, labels= val_labels_elec)

val_loader_humans = DataLoader(val_dataset_human, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader_elec = DataLoader(val_dataset_elec, batch_size=batch_size, shuffle=True, num_workers=1)


In [8]:
num_channels = val_images.shape[1]
num_classes = len(np.unique(val_labels_human))
model = torch.hub.load('moskomule/senet.pytorch','se_resnet20', num_classes=num_classes)
model = update_model_layers(model, num_channels, num_classes)

model.load_state_dict(torch.load('dual_split/model:se_resnet20_epoch:3_lr:0.005_mom:0.9_step:15_gamma:0.3_valloss:1.01_f1loss:0.7_batchsize:64_binary:humans.pt'))


Using cache found in /home/mcs9834/.cache/torch/hub/moskomule_senet.pytorch_master


<All keys matched successfully>

In [9]:
model.eval()
validation_loss = 0
correct = 0
y_true = []
y_pred = []
for data, target in val_loader_humans:
    y_true.append(target)
    #change data sizing here if it comes in in the correct size
    output = model(data)#.view(batch_size, 98, 50, 50)
    validation_loss += F.cross_entropy(output, target, reduction='sum')
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    y_pred.append(torch.flatten(pred))
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

# change number of classes if switching to binary classification
y_true = torch.cat(y_true)
y_pred = torch.cat(y_pred)
f1 = multiclass_f1_score(y_true, y_pred, average='weighted', num_classes = num_classes)
validation_loss /= len(val_loader_humans.dataset)
print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    validation_loss, correct, len(val_loader_humans.dataset),
    100. * correct / len(val_loader_humans.dataset)))
print('val_loss', validation_loss.item(), 'f1_loss:', f1.item())


Validation set: Average loss: 1.0065, Accuracy: 2685/4864 (55%)

val_loss 1.0065159797668457 f1_loss: 0.7006344795227051


In [10]:
y_pred_humans_dual = y_pred.detach().clone()
y_pred_humans_dual

tensor([1, 1, 1,  ..., 1, 1, 1])

In [12]:
# Bring in model
num_classes = len(np.unique(val_labels_elec))
num_channels = val_images.shape[1]
model_elec = torch.hub.load('moskomule/senet.pytorch','se_resnet20', num_classes=num_classes)
model_elec = update_model_layers(model, num_channels, num_classes)

model_elec.load_state_dict(torch.load('model:se_resnet20_epoch:1_lr:0.03_mom:0.9_step:15_gamma:0.3_valloss:0.08_f1loss:0.99_batchsize:32_binary:elec.pt'))


Using cache found in /home/mcs9834/.cache/torch/hub/moskomule_senet.pytorch_master


<All keys matched successfully>

In [13]:
model_elec.eval()
validation_loss = 0
correct = 0
y_true = []
y_pred = []
for data, target in val_loader_elec:
    y_true.append(target)
    #change data sizing here if it comes in in the correct size
    output = model_elec(data)#.view(batch_size, 98, 50, 50)
    validation_loss += F.cross_entropy(output, target, reduction='sum')
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    y_pred.append(torch.flatten(pred))
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

# change number of classes if switching to binary classification
y_true = torch.cat(y_true)
y_pred = torch.cat(y_pred)
f1 = multiclass_f1_score(y_true, y_pred, average='weighted', num_classes = num_classes)
validation_loss /= len(val_loader_elec.dataset)
print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    validation_loss, correct, len(val_loader_elec.dataset),
    100. * correct / len(val_loader_elec.dataset)))
print('val_loss', validation_loss.item(), 'f1_loss:', f1.item())


Validation set: Average loss: 0.0801, Accuracy: 4789/4864 (98%)

val_loss 0.0800526887178421 f1_loss: 0.9922304153442383


In [14]:
y_pred_elec_dual = y_pred.detach().clone()
y_pred_elec_dual

tensor([0, 0, 0,  ..., 0, 0, 0])

In [18]:
y_pred_dual = np.zeros(len(y_pred_humans_dual))
y_pred_dual.fill(5) #making these 5 to ensure only 0s and 1s get filled and not left over

for i in range(len((y_pred_humans_dual))): 
    if y_pred_humans_dual[i] == 1 and y_pred_elec_dual[i] == 0: 
        y_pred_dual[i] = 0
    elif y_pred_humans_dual[i] == 0 and y_pred_elec_dual[i] == 0:
        y_pred_dual[i] = 1
    elif y_pred_humans_dual[i] == 1 and y_pred_elec_dual[i] == 1:
        y_pred_dual[i] = 2
    elif y_pred_humans_dual[i] == 0 and y_pred_elec_dual[i] == 1:
        y_pred_dual[i] = 3


In [19]:
# Need 2 sats val loader to check for comparison
val_labels_all = torch.from_numpy(np.load('val_labels2.npz')['labels'])
val_labels_all

tensor([1, 1, 1,  ..., 0, 1, 1])

In [20]:
y_pred_dual = torch.from_numpy(y_pred_dual.astype('int64'))
y_pred_dual

tensor([0, 0, 0,  ..., 0, 0, 0])

In [21]:
# Num classes = 4
num_classes = 4
f1 = multiclass_f1_score(val_labels_all, y_pred_dual, average='weighted', num_classes = num_classes)
f1.item()

0.6956865787506104

In [ ]:
humans:  model:se_resnet20_epoch:3_lr:0.005_mom:0.9_step:15_gamma:0.3_valloss:1.01_f1loss:0.7_batchsize:64_binary:humans.pt
                                        
elec: model:se_resnet20_epoch:1_lr:0.03_mom:0.9_step:15_gamma:0.3_valloss:0.08_f1loss:0.99_batchsize:32_binary:elec.pt